# Object Detection Pipeline - Hyperparameter Tuning & Model Training

Hyperparameter tuning with Ray Tune on validation set, followed by final training on train+val and evaluation on test set.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-oda/blob/master/notebooks/experiment.ipynb)

In [1]:
# ! pip install dl4cv_oda ray[tune]

In [2]:
import os
import yaml
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)
from ultralytics import YOLO, RTDETR

## Configuration

In [ ]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

IMG_SIZE = 256
EPOCHS = 200
PATIENCE = 20

TUNE_EPOCHS = 30
TUNE_ITERATIONS = 3
GPU_PER_TRIAL = 1

MODELS_TO_TRAIN = {
    "yolov8l": "yolov8l.pt",
    "yolo12l": "yolo12l.pt",
    "rtdetr-l": "rtdetr-l.pt"
}

In [4]:
import requests
if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded {OSM_FILE}")
else:
    print("OSM data ready")

OSM data ready


## Step 1: Clean OSM Data

In [5]:
if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Processed {count} trees")
else:
    print("Cleaned data ready")

Cleaned data ready


## Download Tiles

In [6]:
if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Tiles downloaded")

## Clip Labels & Convert to YOLO Format

In [7]:
if TILES_FILE.exists() and not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"{stats['processed']} tiles, {stats['total_trees']} trees")
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Class mapping: {class_mapping}")
else:
    print("YOLO format data ready")

YOLO format data ready


## Create Train/Val/Test Split

In [8]:
if not (YOLO_DIR / "train").exists():
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)
    print(f"Train: {train_count} | Val: {val_count} | Test: {test_count}")
else:
    print("Train/Val/Test split ready")

config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
config_file_abs = os.path.abspath(config_file)
print(f"Config: {config_file_abs}")

Train/Val/Test split ready
Config: /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml


## Hyperparameter Tuning with Ray Tune

Tune all models with consistent parameters.

In [9]:
tune_results = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"Tuning {model_name}...")
    model = RTDETR(model_weights) if "rtdetr" in model_name.lower() else YOLO(model_weights)
    result_grid = model.tune(data=config_file_abs, epochs=TUNE_EPOCHS, imgsz=IMG_SIZE, 
                            patience=PATIENCE, iterations=TUNE_ITERATIONS, 
                            gpu_per_trial=GPU_PER_TRIAL, use_ray=True)
    tune_results[model_name] = result_grid
    print(f"{model_name} tuning complete")

(_tune pid=3493751) New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=3493751) Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
(_tune pid=3493751) engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.39959679289670946, box=0.07233116412158441, cache=False, cfg=None, classes=None, close_mosaic=10, cls=2.3175763913787866, compile=False, conf=None, copy_paste=0.47809075905939646, copy_paste_mode=flip, cos_lr=False, cutmix=0.5789229641371096, data=/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml, degrees=39.27233571656042, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.8657674304258716, flipud=0.7891965963650895, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.04927335701829

(_tune pid=3493751) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
(_tune pid=3493751)   return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/30      4.32G      1.665     0.8626     0.6309        390        256: 0% ──────────── 0/28  2.1s
       1/30      4.32G      1.706     0.9235     0.6514        329        256: 4% ──────────── 1/28 1.5it/s 2.3s<17.5s
       1/30      4.32G      1.687      0.957     0.6272        314        256: 7% ╸─────────── 2/28 2.7it/s 2.5s<9.7s
       1/30      4.32G      1.666      1.005     0.6123        281        256: 11% ━─────────── 3/28 3.5it/s 2.7s<7.1s
       1/30      4.32G      1.637      1.059     0.6085        255        256: 14% ━╸────────── 4/28 4.4it/s 2.9s<5.5s
       1/30      4.44G      1.647      1.014     0.6164        432        256: 18% ━━────────── 5/28 4.7it/s 3.1s<4.9s
       1/30      5.34G      1.629      0.995     0.6077        339        256: 21% ━━╸───────── 6/28 5.1it/s 3.2s<4.4s
       1/30      5.34G      1.611     0.9584     0.5972        382        256: 25% ━━━───────── 7/28 5.5it/s 3.4s<3.8s
       1/30      5.34G      1.595     0.9527     0.5938       

(_tune pid=3493751) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_tune pid=3493751)   _log_deprecation_warning(
(_tune pid=3493751) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
(_tune pid=3493751)   return Variable

(_tune pid=3493751) 
(_tune pid=3493751)       Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       2/30      7.13G      1.028      0.544     0.2949        396        256: 0% ──────────── 0/28  0.2s
       2/30      7.13G      0.984     0.6198     0.2756        318        256: 4% ──────────── 1/28 2.1it/s 0.3s<12.8s
       2/30      7.13G     0.9796      0.635      0.287        287        256: 7% ╸─────────── 2/28 3.2it/s 0.5s<8.0s
       2/30      7.13G     0.9446     0.6452     0.2712        339        256: 11% ━─────────── 3/28 4.4it/s 0.6s<5.6s
       2/30      7.13G     0.9288     0.6474     0.2643        301        256: 14% ━╸────────── 4/28 5.0it/s 0.8s<4.8s
       2/30      7.13G     0.9076     0.6527      0.263        294        256: 18% ━━────────── 5/28 5.5it/s 0.9s<4.2s
       2/30      7.13G     0.9135     0.6358     0.2699        580        256: 21% ━━╸───────── 6/28 5.6it/s 1.1s<4.0s
       2/30      7.13G     0.8924     0.6426     0.2632       

...

(_tune pid=3523249) 
(_tune pid=3523249) 25 epochs completed in 0.047 hours.
(_tune pid=3523249) Optimizer stripped from /tmp/ray/session_2026-01-11_01-10-16_249916_3452286/artifacts/2026-01-11_01-10-17/tune16/working_dirs/_tune_e917b_00002/runs/detect/tune/weights/last.pt, 66.2MB
(_tune pid=3523249) Optimizer stripped from /tmp/ray/session_2026-01-11_01-10-16_249916_3452286/artifacts/2026-01-11_01-10-17/tune16/working_dirs/_tune_e917b_00002/runs/detect/tune/weights/best.pt, 66.2MB
(_tune pid=3523249) 
(_tune pid=3523249) Validating /tmp/ray/session_2026-01-11_01-10-16_249916_3452286/artifacts/2026-01-11_01-10-17/tune16/working_dirs/_tune_e917b_00002/runs/detect/tune/weights/best.pt...
(_tune pid=3523249) Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
(_tune pid=3523249) rt-detr-l summary: 310 layers, 31,985,795 parameters, 0 gradients, 103.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50

(_tune pid=3523249) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_tune pid=3523249)   _log_deprecation_warning(


(_tune pid=3523249)                    all        188       5167      0.741      0.684      0.657      0.292
(_tune pid=3523249) Speed: 0.0ms preprocess, 8.0ms inference, 0.0ms loss, 2.6ms postprocess per image
(_tune pid=3523249) Results saved to /tmp/ray/session_2026-01-11_01-10-16_249916_3452286/artifacts/2026-01-11_01-10-17/tune16/working_dirs/_tune_e917b_00002/runs/detect/tune


2026-01-11 01:19:01,196	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/detect/tune16' in 0.0022s.
2026-01-11 01:19:01,199	INFO tune.py:1041 -- Total run time: 523.47 seconds (523.46 seconds for the tuning loop).


rtdetr-l tuning complete


## Analyze Results & Extract Best Configs

In [10]:
def analyze_and_save_results(result_grid, model_name):
    results_list = []
    for i, result in enumerate(result_grid):
        df = result.metrics_dataframe
        p = df['metrics/precision(B)'].iloc[-1] if 'metrics/precision(B)' in df else 0
        r = df['metrics/recall(B)'].iloc[-1] if 'metrics/recall(B)' in df else 0
        m = df['metrics/mAP50(B)'].iloc[-1] if 'metrics/mAP50(B)' in df else 0
        f1 = 2 * (p * r) / (p + r + 1e-6) if (p + r) > 0 else 0
        results_list.append({'trial': i, 'config': result.config, 'precision': p, 'recall': r, 'mAP50': m, 'f1': f1})
    
    results_df = pd.DataFrame(results_list).sort_values('f1', ascending=False)
    best = results_df.iloc[0]
    best_config = best['config']
    
    with open(YOLO_DIR / f"best_config_{model_name}.yaml", 'w') as f:
        yaml.dump(best_config, f, default_flow_style=False)
    
    print(f"{model_name:12s} F1: {best['f1']:.4f} | P: {best['precision']:.4f} | R: {best['recall']:.4f} | mAP50: {best['mAP50']:.4f}")
    return results_df, best_config

best_configs = {}
for model_name, result_grid in tune_results.items():
    _, best_configs[model_name] = analyze_and_save_results(result_grid, model_name)

yolov8l      F1: 0.7267 | P: 0.7623 | R: 0.6942 | mAP50: 0.6540
yolo12l      F1: 0.7405 | P: 0.7832 | R: 0.7022 | mAP50: 0.6770
rtdetr-l     F1: 0.7114 | P: 0.7408 | R: 0.6843 | mAP50: 0.6565


## Merge Train+Val for Final Training

In [11]:
trainval_dir = YOLO_DIR / "trainval"
trainval_config = YOLO_DIR / "data_trainval.yaml"

if not trainval_dir.exists():
    trainval_dir.mkdir(exist_ok=True)
    for split in ["train", "val"]:
        for f in (YOLO_DIR / split).glob("*"):
            shutil.copy(f, trainval_dir / f.name)
    with open(config_file, 'r') as f:
        trainval_config.write_text(f.read().replace("train: train", "train: trainval"))
    print(f"Merged train+val into {trainval_dir}")
else:
    print("Train+val merge ready")

Train+val merge ready


## Final Training with Best Hyperparameters

In [12]:
trained_models = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"\nTraining {model_name} on train+val...")
    model = RTDETR(model_weights) if "rtdetr" in model_weights.lower() else YOLO(model_weights)
    cfg = best_configs[model_name].copy()
    cfg.update({'data': str(trainval_config), 'epochs': EPOCHS, 'imgsz': IMG_SIZE, 
                'patience': PATIENCE, 'name': f"final_{model_name}", 'plots': True})
    model.train(**cfg)
    trained_models[model_name] = model
    print(f"{model_name} complete")


Training yolov8l on train+val...
New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.17136807510635188, box=0.09069458705922955, cache=False, cfg=None, classes=None, close_mosaic=10, cls=2.1561602258069144, compile=False, conf=None, copy_paste=0.11720219999800607, copy_paste_mode=flip, cos_lr=False, cutmix=0.4148988700184312, data=../data/yolo/data_trainval.yaml, degrees=21.99565017189849, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.46011086516056865, flipud=0.5554203599910199, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.024565257516903083, hsv_s=0.2583015625177827, hsv_v=0.6205804467605074, imgsz=256, int8=F

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/200        11G      1.722     0.3082     0.6503        486        256: 100% ━━━━━━━━━━━━ 29/29 3.1it/s 9.5s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 4.4it/s 1.4s0.1s
                   all        188       5167     0.0139     0.0453    0.00508   0.000932

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      2/200      6.87G      1.408     0.3608     0.4562        885        256: 3% ──────────── 1/29 1.6it/s 0.4s<17.5s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      2/200      12.5G      1.262     0.4005     0.3636        519        256: 100% ━━━━━━━━━━━━ 29/29 5.2it/s 5.6s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 13.7it/s 0.4s.3s
                   all        188       5167      0.447      0.385      0.315     0.0618

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      3/200      7.04G      1.102     0.4528     0.2845        634        256: 3% ──────────── 1/29 1.7it/s 0.4s<16.0s

...

     43/200      7.95G     0.8604     0.4722     0.1887        542        256: 100% ━━━━━━━━━━━━ 29/29 5.6it/s 5.2s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 13.8it/s 0.4s.3s
                   all        188       5167      0.744      0.704      0.666      0.287

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
     44/200      7.37G     0.8906     0.4585      0.192       1208        256: 0% ──────────── 0/29  0.2s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     44/200      13.9G     0.8708     0.4657     0.1848        375        256: 100% ━━━━━━━━━━━━ 29/29 5.3it/s 5.5s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 13.6it/s 0.4s.3s
                   all        188       5167      0.754      0.711       0.67      0.277

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
     45/200      7.79G     0.7833      0.489     0.1992        761        256: 3% ──────────── 1/29 1.6it/s 0.3s<17.1s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     45/200      10.3G     0.8553     0.4727     0.1887        321        256: 100% ━━━━━━━━━━━━ 29/29 5.5it/s 5.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 13.5it/s 0.4s.3s
                   all        188       5167      0.762      0.726      0.692      0.294
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 35, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

45 epochs completed in 0.082 hours.
Optimizer stripped from /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/detect/final_rtdetr-l2/weights/last.pt, 66.2MB
Optimizer stripped from /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/detect/final_rtdetr-l2/weights/best.pt, 66.2MB

Validating /home/krschap/academia/dl4cv-object

## Evaluate on Test Set

In [13]:
test_results = {}
for name, model in trained_models.items():
    print(f"Evaluating {name}...")
    m = model.val(split='test')
    p, r = m.box.mp, m.box.mr
    test_results[name] = {"precision": p, "recall": r, "f1": 2 * (p * r) / (p + r + 1e-6),
                         "mAP50": m.box.map50, "mAP50-95": m.box.map}

Evaluating yolov8l...
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
Model summary (fused): 112 layers, 43,607,379 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5183.3±577.8 MB/s, size: 148.8 KB)
val: Scanning /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/test.cache... 45 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 45/45 180.4Kit/s 0.0s
val: /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/test/OAM-6783-293582-19.png: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 4.3it/s 0.7s0.6s
                   all         45        981      0.679      0.597      0.551      0.216
Speed: 0.3ms preprocess, 8.5ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebo

In [14]:
results_df = pd.DataFrame(test_results).T.sort_values('f1', ascending=False)
print("Test Set Results:")
print(results_df.to_string())
print(f"\nBest: {results_df.index[0]}")

Test Set Results:
          precision    recall        f1     mAP50  mAP50-95
rtdetr-l   0.693237  0.690112  0.691670  0.636407  0.265799
yolov8l    0.678799  0.597350  0.635474  0.550892  0.216421
yolo12l    0.668519  0.602446  0.633765  0.546075  0.218764

Best: rtdetr-l


## F1 Score per Epoch

In [15]:
n = len(MODELS_TO_TRAIN)
cols = min(3, n)
rows = (n + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
axes = axes.flatten() if n > 1 else [axes]

for idx, model_name in enumerate(MODELS_TO_TRAIN.keys()):
    csv = Path(f"runs/detect/final_{model_name}/results.csv")
    if csv.exists():
        df = pd.read_csv(csv)
        df.columns = df.columns.str.strip()
        p, r = df['metrics/precision(B)'], df['metrics/recall(B)']
        f1 = 2 * (p * r) / (p + r + 1e-6)
        axes[idx].plot(f1, label='F1', linewidth=2, color='#2E86AB')
        axes[idx].plot(p, label='Precision', alpha=0.7, color='#A23B72')
        axes[idx].plot(r, label='Recall', alpha=0.7, color='#F18F01')
        axes[idx].set_title(model_name.upper(), fontsize=11, fontweight='bold')
        axes[idx].set_xlabel('Epoch')
        axes[idx].set_ylabel('Score')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)

for idx in range(n, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('f1_scores_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

<Figure size 1800x500 with 3 Axes>